# Final Project: An Improved Attempt at Modeling the Trajectory of a Golf Ball via Curve Fitting using Drag and the Magnus Effect 
## By Will Frondorf

## Abstract
By using the following program, users can see how a hypothetical golf shot will travel given different factors, such as air resistance, drag, wind, weather conditions, and the Magnus Effect. A main part of this program is seeing how wind impacts the trajectory of the ball, particularly at higher altitudes and larger spinrates. This program servers as an investiagtion into the Magnus Effect and the impact it has on spinning objects in flight. 

## An Overview of the Model
This Python program makes use of the Runge-Kutta 4 formula, as well as the Magnus Force equation itterated over multiple timesteps. This computational model will take the entered data and plug it into the drag and lift formulas for a golf ball. The drag coefficient of a golf ball is $C_d=\frac{1}{4}(1+\frac{1}{1+e^{2v-12}})$ where $v$ is the velocity of the ball in meters per second. The lift coefficient of a golf ball is $C_L=0.55 x S^{0.4}$ where $S$ is the spinrate (the amount of times the ball completes a single rotation per second) of the golf ball. By combining these factors, you can see how the ball moves. Additionally, this program implements the Magnus Effect, which states that a force is exherted on a spinning sphere in flight that deflects the sphere in a direction perpindicular to its spin. For example, a ball with backspin will be deflected upward and a ball with topspin will be deflected downward. The Magnus Force can be calulated using the following formula: $F_{magnus}=\alpha(\omega \times v)$ where $(\omega \times v)$ is the cross product of the sphere's rotation $\omega$ and velocity $v$ multiplied by the constant $\alpha$, where $\alpha=\frac{1}{2}C_L\frac{\rho Ar}{S}$.

#### Calling up Required Packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ode
import random
from IPython.display import Markdown, display
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from mpl_toolkits.mplot3d import Axes3D

#### Setting the figures to be Notebooks, allowing users to manipulate the figures via zooming, panning, etc.

In [ ]:
%config InlineBackend.figure_format="retina"
%matplotlib notebook

#### Creating the Various Functions that will be Used throughout the Program

In [ ]:
def mag(v):
    #Calculate magnitude of an array
    return np.sqrt(np.dot(v,v))

def hat(v):
    #Calculate unit vector of an array
    return v/mag(v)

def cross(v1,v2):
    #Calculate the cross product of two vectors
    return np.cross(v1,v2)

def getCd(v):
    #Calculate value of drag coefficient for a particular speed and case
    
    Cd=0.25*(1+1/(1+np.exp(2*v-12))) #Drag Coefficient of a Golf Ball
    return Cd

def model_magnus(d,tn):
    #Magnus force function
    #Data
    x=d[0]
    y=d[1]
    z=d[2]  
    vx=d[3]
    vy=d[4]
    vz=d[5]
    
    #Derivatives
    rate=np.zeros(6) 
    rate[0]=vx-vwindx
    rate[1]=vy-vwindy
    rate[2]=vz-vwindz
    
    #Speed
    v=np.array([vx,vy,vz])
    vmag=mag(v)
    
    #Calculate force and dv/dt
    Cd=getCd(vmag)
    b2=1/2*Cd*rho*A
    
    S=r*omegamag/vmag
    CL=0.55*S**0.4           #Lift coefficient for a golf ball
    alpha=1/2*CL*(rho*A*r/S)

    FM=alpha*cross(omega,v) #Magnus Force
    
    rate[3]=-b2*vmag*vx/m+FM[0]/m       #Change in x-axis
    rate[4]=(-b2*vmag*vy-m*g)/m+FM[1]/m #Change in y-axis
    rate[5]=(-b2*vmag*vz)/m+FM[2]/m     #Change in z-axis
    
    return rate

def run_magnus(data,omega): 
    global b2, alpha, t
    
    v=data[3:6]
    vmag=mag(v)

    #Storing Trajectory
    traj=np.zeros((Nsteps, 4)) #store t, x, y, z for plotting
    traj[0,:]=np.array([t, x0, y0, z0]) #store initial time and position

    for n in range(0,Nsteps-1):

        #Update data
        data=ode.RK4(model_magnus, data, t, h)

        #Update t
        t=t+h

        #Store data for plotting
        traj[n+1,:]=np.array([t, data[0], data[1], data[2]])
    
    return traj

#### Establishing the Various Parameters and Constants that will be Used throughout the Program

In [ ]:
#Golf Ball Parameters------------------------------------------------------------------------------------------------------
d=0.0426     #Diameter of the ball, meters (4.267cm diameter)
r=d/2        #Radius of golf ball, meters
A=np.pi*r**2 #Cross sectional area of golf ball
m=0.04593    #Mass of golf ball, kg (45.93g mass)

#Environment Parameters----------------------------------------------------------------------------------------------------
g=9.8           #Gravity, N/kg
rho=1.2754      #Density of Air, kg/m^3
mu=1.81e-5      #Viscosity of Air, kg/m/s
Cd=0            #Initial Drag Coefficient, will change as model is made
b2=1/2*Cd*rho*A #Quadratic Drag Equation

S=0.01                 #Spin Parameter, will change as omega and v change
CL=0                   #Initial Lift Coefficient, will change with S
alpha=1/2*CL*rho*A*r/S

#Setting up Time-----------------------------------------------------------------------------------------------------------
t=0               #Initial Time       
h=0.01            #Timestep Value
Nsteps=int(10/h)  #Number of Itterations

## The T.I.T.U.S. Golf Shot Simulator, Mk. II
Users can utilize this code to plot their golf shots. By utilizing the dropdown menus and sliders below, users can see how their golf shots would behave in real life. Users can edit the initial speed of the ball, launch angles, wind speeds, and much more. After all the input data is read in, both a 2-dimensional graph and a 3-dimensional model are generated showing the trajectory of the golf ball with the user's choice of spin. Alongside the plots, additional data is printed beneath the plots, such as maximum distances and wind speeds. All input data is entered in imperial units (mph, yards, etc.), converted into metric (radians, meters per second, etc.) for all the calculations, and then reconverted into the user's choice of output unit system, either imperial or metric. All output data, including the axes of the plots, are converted into the unit system the user chose in the menu below.

Here is a table showing ranges of typical swing speeds, launch angles, and spinrates of golf shots taken with each of the clubs avaiable to use in the T.I.T.U.S., Mk. II. This is here to help you make a better decision of what type of parameters you should use for the manual input menus if you wish to model an accurate golf shot with real data instead of a wacky, extreme case situation. Remember, this table is meerly a suggestion based on realistic data. Feel free to completely ignore this and try whatever numbers come to mind. This table is adapted from an already-existing table found on the TrackMan Golf website (link in the References section).

| Club | Swing Speed (mph, m/s) | Spinrate (rpm) |
| :-: | :-: | :-: |
| Driver | 110-130, 49-58 | 2,400-3,200 |
| 3-Wood | 100-110, 44-49 | 3,300-3,800 |
| 3-Iron | 96-100, 42-44 | 4,400-4,700 |
| 4-Iron | 94-98, 42-43 |  4,700-4,900 |
| 5-Iron | 92-96, 41-43 | 5,000-5,500 |
| 6-Iron | 90-94, 40-42 | 6,000-6,800 |
| 7-Iron | 88-92, 39-41 | 6,800-7,200 |
| 8-Iron | 85-89, 38-40 | 7,700-8,100 |
| 9-Iron | 83-87, 37-39 | 8,400-8,800 |
| Sand Wedge | 82-86, 37-38 | 8,800-9,100 |
| Pitching Wedge | 81-85, 32-38 | 9,000-9,500 |
| Putter | 5-15, 2-7 | 10-30 |

#### Creating the User Input for all the Manually Input Data

In [ ]:
#User Input Dropdown Menus and Sliders
display(Markdown('**<center><span style="color:green">Welcome to the Totally Intense Titalist Utilizing System, Mk. II! You can use this simulator to model how your golf shot will \nfly through the air based on a set of manually input parameters. Please use the following to determine some required \ninformation about your modeled golf shot. Have fun and I hope you enjoy!</span></center>**'))
print()

#Starting Information Inputs-----------------------------------------------------------------------------------------------
print(u"\u001b[32;1m\u001b[4mSTARTING INFORMATION                                                                                                                        \u001b[0m")

#Selection for what Type of Golf Club being Used in the Model 
print("\033[1m\033[4mClub Selection\033[0m\033[0m")
print("Select which golf club you are using for this shot.")
clubmenu=widgets.Dropdown(options=['Driver', '3-Wood', '3-Iron', '4-Iron', '5-Iron', '6-Iron', '7-Iron', '8-Iron', '9-Iron', 'Sand Wedge', 'Pitching Wedge', 'Putter'])
display(clubmenu)

#Selection for the Unit System the figures will use when Outputting Data
print("\033[1m\033[4mUnit Selection\033[0m\033[0m")
print("Select which system of units you want all your output data presented in. Imperial uses yards for distance and miles per hour \nfor velocity. Metric uses meters for distance and meters per second for velocity.")
unitmenu=widgets.Dropdown(options=['Imperial', 'Metric'])
display(unitmenu)

#Selection for Showing the Location of the Hole on the 2D Plot
print("\033[1m\033[4mShow Pin Selection\033[0m\033[0m")
print("Select if you want to show where the hole is located on the 2D plot.")
holemenu=widgets.Dropdown(options=['No', 'Yes'])
display(holemenu)

#Selection for Choosing how far away the Hole on the Green is
print("\033[1m\033[4mPin Distance Selection\033[0m\033[0m")
print("If you selected to show the location of the hole on the 2D plot, select how far away (in yards) the hole is.")
holeslider=widgets.FloatSlider(min=1, max=500, step=1, value=250, readout_format='.0f');
display(holeslider)

#Selection to Save the 2D Plot as a PNG
print("\033[1m\033[4mSave Selection\033[0m\033[0m")
print("Select if you wish to save the 2-Dimensional model as a PNG file after it is made.")
savemenu=widgets.Dropdown(options=['No', 'Yes'])
display(savemenu)

print()

#Spinrate Information Inputs-----------------------------------------------------------------------------------------------
print(u"\u001b[32;1m\u001b[4mSPINRATE INFORMATION                                                                                                                  \u001b[0m")

#Selection for the Type of Spin being Modeled
print("\033[1m\033[4mSpin Type Selection\033[0m\033[0m")
print("Select the type of spin you wish to model in this shot.")
spinmenu=widgets.Dropdown(options=['Backspin/Topspin', 'Sidespin', 'Spiral'])
display(spinmenu)

#Selection for if the Model uses a Random Spinrate or a Manually Input One
print("\033[1m\033[4mType of Spinrate Selection\033[0m\033[0m")
print("Select which type of spinrate you will be using for this shot. If you choose to use a randomly generated one, you can simply \nignore the slider below. If you wish to enter your own value for the spinrate of the golf ball, use the slider below.")
ratemenu=widgets.Dropdown(options=['Random Spinrate', 'Manually Selected Spinrate'])
display(ratemenu)

#Selection for a Manual Spinrate
print("\033[1m\033[4mManual Spinrate Selection\033[0m\033[0m")
print("Select the amount of times the golf ball spins (in rotations per minute) right after you hit it.")
spinslider=widgets.IntSlider(min=10, max=9500, step=1, value=4755);
display(spinslider)

print()

#Terrain Information Inputs------------------------------------------------------------------------------------------------
print(u"\u001b[32;1m\u001b[4mTERRAIN INFORMATION                                                                                                                 \u001b[0m")

#Selection for what Type of Terrain the Shot is being Hit from
print("\033[1m\033[4mTerrain Selection\033[0m\033[0m")
print("Select the type of terrain your are hitting off of.")
terrainmenu=widgets.Dropdown(options=['Tee Box', 'Fairway', 'Rough', 'Green', 'Bunker'])
display(terrainmenu)

#Selection to choose a starting height the User is Standing at when they take the Shot
print("\033[1m\033[4mElevation Selection\033[0m\033[0m")
print("Select the elevation (in feet) that you standing at when you hit the ball.")
elevationslider=widgets.FloatSlider(min=0, max=30, step=0.1, value=0,readout_format='.1f');
display(elevationslider)

print()

#Swing Information Inputs-------------------------------------------------------------------------------------------
print(u"\u001b[32;1m\u001b[4mSWING INFORMATION                                                                                                                        \u001b[0m")

#Selection for the Vertical Launch Angle of the Shot
print("\033[1m\033[4mY-axis Launch Angle Selection\033[0m\033[0m")
print("Select the vertical angle (in degrees) that your ball has right after you hit it.")
UDangleslider=widgets.FloatSlider(min=5, max=85, step=0.1, value=45, readout_format='.1f');
display(UDangleslider)

#Selection for the Horizontal Launch Angle of the Shot
print("\033[1m\033[4mX-axis Launch Angle Selection\033[0m\033[0m")
print("Select the horizontal (in degrees) that your ball has right after you hit it. Negative numbers indicate an angle to the left \nwhile positive numbers indicate an angle to the right.")
LRangleslider=widgets.FloatSlider(min=-90, max=90, step=0.1, value=0, readout_format='.1f');
display(LRangleslider)

#Selection for the Speed of the Swing
print("\033[1m\033[4mSwing Speed Selection\033[0m\033[0m")
print("Select the speed (in mph) of your swing.")
swingslider=widgets.FloatSlider(min=2, max=120, step=0.1, value=61, readout_format='.1f');
display(swingslider)

print()

#Wind Speed and Weather Information Inputs---------------------------------------------------------------------------------------------
print(u"\u001b[32;1m\u001b[4mWIND SPEED AND WEATHER INFORMATION                                                                                                                  \u001b[0m")

#Selection for Front-Back Wind Speeds
print("\033[1m\033[4mX-axis Wind Speed Selection\033[0m\033[0m")
print("Select the speed (in mph) of the wind blowing forwards or backwards. Negative numbers indicate wind blowing forwards while \npositive numbers indicate wind blowing backwards.")
FBwindslider=widgets.FloatSlider(min=-25, max=25, step=0.1, value=0, readout_format='.1f');
display(FBwindslider)

#Selection for Up-Down Wind Speeds
print("\033[1m\033[4mY-axis Wind Speed Selection\033[0m\033[0m")
print("Select the speed (in mph) of the wind blowing up or down. Negative numbers indicate wind blowing straight up while positive \nnumbers indicate wind blowing straight down.")
UDwindslider=widgets.FloatSlider(min=-25, max=25, step=0.1, value=0, readout_format='.1f');
display(UDwindslider)

#Selection for Left-Right Wind Speeds
print("\033[1m\033[4mZ-axis Wind Speed Selection\033[0m\033[0m")
print("Select the speed (in mph) of the wind blowing left or right. Negative numbers indicate wind blowing right while positive \nnumbers indicate wind blowing left.")
LRwindslider=widgets.FloatSlider(min=-25, max=25, step=0.1, value=0, readout_format='.1f');
display(LRwindslider)

#Selection for the Type of Weather the User is Playing Golf in
print("\033[1m\033[4mWeather Selection\033[0m\033[0m")
print("Select the time of weather you are playing in.")
weathermenu=widgets.Dropdown(options=['Sunny and Calm', 'Sunny and Humid', 'Sunny and Windy', 'Overcast and Windy', 'Thunderstorm', 'Blizzard'])
display(weathermenu)

#### Assigning Various Conditionals based on the User's Input Data

In [ ]:
#Menu and Slider Option Conditionals
#Club Selection Options----------------------------------------------------------------------------------------------------
#Random Spinrate Parameters
if ratemenu.value=='Random Spinrate':
    if clubmenu.value=='Driver':
        clubname='Driver'                  #Name of golf club
        spinrate=random.randint(2400,3200) #Spinrate of golf ball
    if clubmenu.value=='3-Wood':
        clubname='3-Wood'                  #Name of golf club#Name of golf club
        spinrate=random.randint(3300,3800) #Spinrate of golf ball
    if clubmenu.value=='3-Iron':
        clubname='3-Iron'                  #Name of golf club
        spinrate=random.randint(4400,4700) #Spinrate of golf ball
    if clubmenu.value=='4-Iron':
        clubname='4-Iron'                  #Name of golf club
        spinrate=random.randint(4701,4900) #Spinrate of golf ball
    if clubmenu.value=='5-Iron':
        clubname='5-Iron'                  #Name of golf club
        spinrate=random.randint(5000,5500) #Spinrate of golf ball
    if clubmenu.value=='6-Iron':
        clubname='6-Iron'                  #Name of golf club
        spinrate=random.randint(6000,6800) #Spinrate of golf ball
    if clubmenu.value=='7-Iron':
        clubname='7-Iron'                  #Name of golf club
        spinrate=random.randint(6800,7200) #Spinrate of golf ball
    if clubmenu.value=='8-Iron':
        clubname='8-Iron'                  #Name of golf club
        spinrate=random.randint(7700,8100) #Spinrate of golf ball
    if clubmenu.value=='9-Iron':
        clubname='9-Iron'                  #Name of golf club
        spinrate=random.randint(8400,8800) #Spinrate of golf ball
    if clubmenu.value=='Sand Wedge':
        clubname='Sand Wedge'              #Name of golf club
        spinrate=random.randint(8800,9100) #Spinrate of golf ball
    if clubmenu.value=='Pitching Wedge':
        clubname='Pitching Wedge'          #Name of golf club
        spinrate=random.randint(9000,9500) #Spinrate of golf ball
    if clubmenu.value=='Putter':
        clubname='Putter'              #Name of golf club
        spinrate=random.randint(10,30) #Spinrate of golf ball

#Manual Spinrate Parameters
if ratemenu.value=='Manually Selected Spinrate':
    spinrate=spinslider.value
    
    if clubmenu.value=='Driver':
        clubname='Driver'                #Name of golf club
    if clubmenu.value=='3-Wood':
        clubname='3-Wood'                #Name of golf club
    if clubmenu.value=='3-Iron':
        clubname='3-Iron'                #Name of golf club
    if clubmenu.value=='4-Iron':
        clubname='4-Iron'                #Name of golf club
    if clubmenu.value=='5-Iron':
        clubname='5-Iron'                #Name of golf club
    if clubmenu.value=='6-Iron':
        clubname='6-Iron'                #Name of golf club
    if clubmenu.value=='7-Iron':
        clubname='7-Iron'                #Name of golf club
    if clubmenu.value=='8-Iron':
        clubname='8-Iron'                #Name of golf club
    if clubmenu.value=='9-Iron':
        clubname='9-Iron'                #Name of golf club
    if clubmenu.value=='Sand Wedge':
        clubname='Sand Wedge'            #Name of golf club
    if clubmenu.value=='Pitching Wedge':
        clubname='Pitching Wedge'        #Name of golf club
    if clubmenu.value=='Putter':
        clubname='Putter'                #Name of golf club

#Unit System Conversion Factors and Labels for the Axes of the Plots-------------------------------------------------------
#Conversion Factors and Labels if the chosen System was Imperial
if unitmenu.value=='Imperial':
    unit="yards"                #Name of distance unit, used for printouts of distance
    unitvelocity="mph"          #Name of velocity unit, used for printouts of speed
    velocityconversion=1        #Conversion factor that converts the input velocity of mph to mph
    unitconversion=1.094        #Conversion factor that converts the input imperial units to metric units
    holeconversion=1            #Conversion factor that converts the input distance to the hole in yards to the chosen unit output
    yardconversion=1.094        #Conversion factor that converts the input distance of yards to meters to calculate remaining distance to the hole
    unitlabelx='CARRY (yards)'  #Label of X-axis on plot
    unitlabely='HEIGHT (yards)' #Label of Y-axis on plot
    unitlabelz='CURVE (yards)'  #Label of Z-axis on plot
    
#Conversion Factors and Labels if the chosen System was Metric
if unitmenu.value=='Metric':
    unit="meters"                    #Name of distance unit, used for printouts of distance
    unitvelocity="meters per second" #Name of velocity unit, used for printouts of speed
    velocityconversion=2.237         #Conversion factor that converts the input velocity of mph to meters per second
    unitconversion=1                 #Conversion factor that converts the calculated metric units to metric units
    holeconversion=1.094             #Conversion factor that converts the input distance to the hole in yards to meters
    yardconversion=1                 #Conversion factor that converts the calculated distance of meters to calculate remaining distance to the hole
    unitlabelx='CARRY (meters)'      #Label of X-axis on plot
    unitlabely='HEIGHT (meters)'     #Label of Y-axis on plot
    unitlabelz='CURVE (meters)'      #Label of Z-axis on plot
        
#Terrain Impact and Color Options------------------------------------------------------------------------------------------
#Terrain Impact will give a slight variation to the intitial velcoity and direction of the shot to simulate 
#hitting the ball on different surfaces. The chosen terrain type will also change the color of the 3D map to match the 
#surface you are hitting off of

#Colors if the Terrain was the Tee Box
if terrainmenu.value=='Tee Box':
    terrainname="Tee Box"     #Assigns the name of the terrain so it can be output in the titles of the plots
    terrainimpact=10          #Terrain Impact is added to the initial veolicty of the golf ball
    bordercolor='forestgreen' #Changes the color of the border around the two figures
    labelcolor='white'        #Changes the color of the title, axis labels, and tickmarks of the two figures
    mapcolor='forestgreen'    #Changes the color of the ground of the 3D plot to show what terrain you are hitting off of
    
#Colors if the Terrain was the Fairway
if terrainmenu.value=='Fairway':
    terrainname="Fairway"
    terrainimpact=7
    bordercolor='green'
    labelcolor='white'
    mapcolor='green'
    
#Colors if the Terrain was the Rough
if terrainmenu.value=='Rough':
    terrainname="Rough"
    terrainimpact=-3
    bordercolor='darkgreen'
    labelcolor='white'
    mapcolor='darkgreen'
    
#Colors if the Terrain was the Green
if terrainmenu.value=='Green':
    terrainname="Green"
    terrainimpact=0
    bordercolor='lime'
    labelcolor='black'
    mapcolor='lime'
    
#Colors if the Terrain was the Bunker
if terrainmenu.value=='Bunker':
    terrainname="Bunker"
    terrainimpact=-7
    bordercolor='khaki'
    labelcolor='black'
    mapcolor='khaki'

#Assigning the Strength of the Wind based on the User's Input--------------------------------------------------------------
vwindx=FBwindslider.value #Wind in X-Axis
vwindy=UDwindslider.value #Wind in Y-Axis
vwindz=LRwindslider.value #Winf in Z-Axis

#Weather Conditions and Color Options--------------------------------------------------------------------------------------
#The weatherimpact variable is added to the initial velocity of the golf ball. It changes based on the type of weather
#chosen as a way to simulate hitting the golf ball in different weather conditions (for example, the ball travels farther
#in humid air). The weatherspeed variable is a multiplier on the value of the user's input wind speed, used to simulate
#higher winds in storms

#Parameters if the Weather is Calm
if weathermenu.value=='Sunny and Calm':
    weathername='Calm Conditions' #Type of weather condition
    planecolor='cyan'             #Color of the sky planes on the 3D plot are changed to match the weather condition
    weatherimpact=1               #weatherimpact value for sunny and calm weather
    weatherspeed=1                #wind speed multiplier for sunny and calm weather
    
#Parameters if the Weather is Humid
if weathermenu.value=='Sunny and Humid':
    weathername='Humid Conditions'
    planecolor='cyan'
    weatherimpact=7
    weatherspeed=1
    
#Parameters if the Weather is Windy
if weathermenu.value=='Sunny and Windy':
    weathername='Windy Conditions'
    planecolor='cyan'
    weatherimpact=5
    weatherspeed=1.5
    
#Parameters if the Weather is Overcast
if weathermenu.value=='Overcast and Windy':
    weathername='Overcast Conditions'
    planecolor='lightgray'
    weatherimpact=4
    weatherspeed=1.5
    
#Parameters if the Weather is a Thunderstorm
if weathermenu.value=='Thunderstorm':
    weathername='a Thunderstorm'
    planecolor='slategray'
    weatherimpact=-7
    weatherspeed=2

#Parameters if the Weather is a Blizzard
if weathermenu.value=='Blizzard':
    weathername='a Blizzard'
    planecolor='powderblue'
    weatherimpact=-5
    weatherspeed=1.2

The cell below is home to all the magic of what this program does. The various conditionals and user input data is combined in order to calculate the various trajectories. First, the wind speed slider data is read in and the direction of wind is determined. Wind speed is multiplied by the weatherspeed parameter to better simulate wind speeds in the user's selected weather conditions. Next, the launch angle data is read in and the degree angles are converted to radians in the rest of the calcuations. Initial velocity is read in from the user's input and the terrain and weather factors are added to the velocity. This is to simulate hitting the golf ball in different situations, such as the sand of a bunker acting as a high-friction source slowing down movement or the rain of a thunderstorm making the ball or clubface wet, thus reducing speed. Initial positions of the ball and initial direction of motion for the ball are then read in. Then, all six types of trajectories are calculated. Each spinrate is calulated and then imported into the Run_Magnus funtion defined earlier. Then each trajectory is calaculated and imported into an array that stores all the data. Lastly, variables are assigned to the maximum values of each trajectory's three position terms to be output later after the plots.

#### Calculating all the Trajectories

In [ ]:
#Begining Message----------------------------------------------------------------------------------------------------------
display(Markdown('**<center><span style="color:green">================================================== PROGRAM BEGINING ==================================================</span></center>**'))

#Wind Information----------------------------------------------------------------------------------------------------------
#Wind in X-Axis
if vwindx>0:
    vwindx=-FBwindslider.value*weatherspeed #Determines that wind is blowing forwards
if vwindx<0:
    vwindx=FBwindslider.value*weatherspeed  #Determines that wind is blowing backwards
if vwindx==0:
    vwindx=0                                #Determines that wind is not blowing

#Wind in Y-Axis
if vwindy>0:
    vwindy=-UDwindslider.value*weatherspeed #Determines that wind is blowing upwards
if vwindy<0:
    vwindy=UDwindslider.value*weatherspeed  #Determines that wind is blowing downwards
if vwindy==0:
    vwindy=0                                #Determines that wind is not blowing

#Wind in Z-Axis
if vwindz>0:
    vwindz=-LRwindslider.value*weatherspeed #Determines that wind is blowing right
if vwindz<0:
    vwindz=LRwindslider.value*weatherspeed  #Determines that wind is blowing left
if vwindz==0:
    vwindz=0                                #Determines that wind is not blowing

#Unit Conversion-----------------------------------------------------------------------------------------------------------
#Vertical Launch Angle
phideg=UDangleslider.value  #User's input for vertical launch angle in degrees
phi=phideg*np.pi/180        #Convert degrees to radians
phi=np.pi/2-phi

#Horizontal Launch Angle
thetadeg=LRangleslider.value #User's input for horizontal launch angle in degrees
theta=thetadeg*np.pi/180     #Convert degrees to radians

#Initial Positions and Velocities------------------------------------------------------------------------------------------
#Intitial Velocity
vmag0mph=(1.5*swingslider.value)+terrainimpact+weatherimpact #Initital velocity of golf ball taken from user's input in mph
vmag0=vmag0mph*0.44704                                       #Convert mph to m/s

#Initial Positions
x0=0
y0=elevationslider.value
z0=0

#Intitial Velocity Directions
vx0=vmag0*np.sin(phi)*np.cos(theta) #Starting velocity and direction in the X-axis
vy0=vmag0*np.cos(phi)               #Starting velocity and direction in the Y-axis
vz0=vmag0*np.sin(phi)*np.sin(theta) #Starting velocity and direction in the Z-axis

#Spinrates-----------------------------------------------------------------------------------------------------------------
#Spinrate and Direction of Spin of Each Type of Spin
omegamag=spinrate*2*np.pi/60     #Spin Parameter (radians/second)
omega1=np.array([0,0,omegamag])  #Backspin
omega2=np.array([0,0,-omegamag]) #Topspin
omega3=np.array([0,-omegamag,0]) #Clockwise Sidespin
omega4=np.array([0,omegamag,0])  #Counterclockwise Sidespin
omega5=np.array([omegamag,0,0])  #Clockwise Spiral
omega6=np.array([-omegamag,0,0]) #Counterclockwise Spiral

#Trajectories for Each Type of Spin----------------------------------------------------------------------------------------
#Array for instantaneous position and velocity data
d=np.array([x0,y0,z0,vx0,vy0,vz0])

#Backspin
omega=omega1               #Assigning the spin in the Magnus Effect formula to be the spin of a golf ball with backspin
trajA=run_magnus(d,omega1) #Running the Magnus Effect formula to get the trajectory of the golf ball
traj1=trajA[trajA[:,2]>=0] #Assigning the trajectory as an array holding the timestep, x-,y-, and z-coordinates

#Topspin
omega=omega2               #Assigning the spin in the Magnus Effect formula to be the spin of a golf ball with topspin
trajB=run_magnus(d,omega2) #Running the Magnus Effect formula to get the trajectory of the golf ball
traj2=trajB[trajB[:,2]>=0] #Assigning the trajectory as an array holding the timestep, x-,y-, and z-coordinates

#Clockwise Sidespin
omega=omega3               #Assigning the spin in the Magnus Effect formula to be the spin of a golf ball with clockwise sidespin
trajC=run_magnus(d,omega3) #Running the Magnus Effect formula to get the trajectory of the golf ball
traj3=trajC[trajC[:,2]>=0] #Assigning the trajectory as an array holding the timestep, x-,y-, and z-coordinates

#Counterclockwise Sidespin
omega=omega4               #Assigning the spin in the Magnus Effect formula to be the spin of a golf ball with counterclockwise sidepsin
trajD=run_magnus(d,omega4) #Running the Magnus Effect formula to get the trajectory of the golf ball
traj4=trajD[trajD[:,2]>=0] #Assigning the trajectory as an array holding the timestep, x-,y-, and z-coordinates

#Clockwise Spiral
omega=omega5               #Assigning the spin in the Magnus Effect formula to be the spin of a golf ball with clockwise spiral
trajE=run_magnus(d,omega5) #Running the Magnus Effect formula to get the trajectory of the golf ball
traj5=trajE[trajE[:,2]>=0] #Assigning the trajectory as an array holding the timestep, x-,y-, and z-coordinates

#Counterclockwise Spiral
omega=omega6               #Assigning the spin in the Magnus Effect formula to be the spin of a golf ball with counterclockwise spiral
trajF=run_magnus(d,omega6) #Running the Magnus Effect formula to get the trajectory of the golf ball
traj6=trajF[trajF[:,2]>=0] #Assigning the trajectory as an array holding the timestep, x-,y-, and z-coordinates

#Maximum Distances across all Type of Spin---------------------------------------------------------------------------------
maxxback=np.max(traj1[:,1])     #max horizontal distance for backspin
maxyback=np.max(traj1[:,2])     #max vertical distance for backspin
maxzback=np.max(traj1[:,3])     #max sideways distance for backspin

maxxtop=np.max(traj2[:,1])      #max horizontal distance for topspin
maxytop=np.max(traj2[:,2])      #max vertical distance for topspin
maxztop=np.max(traj2[:,3])      #max sideways distance for topspin

maxxCLside=np.max(traj3[:,1])   #max horizontal distance for clockwise sidespin
maxyCLside=np.max(traj3[:,2])   #max vertical distance for clockwise sidespin
maxzCLside=np.max(traj3[:,3])   #max sideways distance for clockwise spidespin

maxxCCside=np.max(traj4[:,1])   #max horizontal distance for counterclockwise sidespin
maxyCCside=np.max(traj4[:,2])   #max vertical distance for counterclockwise sidespin
maxzCCside=np.max(traj4[:,3])   #max sideways distance for counterclockwise sidespin

maxxCLspiral=np.max(traj5[:,1]) #max horizontal distance for clockwise spiral
maxyCLspiral=np.max(traj5[:,2]) #max vertical distance for clockwise spiral
maxzCLspiral=np.max(traj5[:,3]) #max sideways distance for clockwise spiral

maxxCCspiral=np.max(traj6[:,1]) #max horizontal distance for counterclockwise spiral
maxyCCspiral=np.max(traj6[:,2]) #max vertical distance for counterclockwise spiral
maxzCCspiral=np.max(traj6[:,3]) #max sideways distance for counterclockwise spiral

#Ending Message------------------------------------------------------------------------------------------------------------
display(Markdown('**<center><span style="color:green">================================================= PROGRAM TERMINATED =================================================</span></center>**'))

Using the cell below, users can generate the two plots mentioned above. The cell that runs all the calculations and the cell that plots all the data are separated on purpose. All the plotting code uses generic commands that rely on conversion factors and preselected color options based on the user's input. Since all those variables are defined in the cell below, users can easily switch to a different type of spin in the dropdown menu above and then run the plotting cell below without running any other cell. This allows for easy switching between spin types on the fly. However, this only works for changing the type of spin. All other factors (unit system, terrain, swing speed, wind speeds, weather, etc.) must be redefined in the Conditionals Cell below the user inputs if they are changed in the dropdown menus and sliders. 

#### Creating the Two Plots that Show Trajectory in both 2D and 3D

In [ ]:
display(Markdown('**<center><span style="color:green">================================================== PROGRAM BEGINING ==================================================</span></center>**'))
#Creating the 2D Plot-----------------------------------------------------------------------------------------------------
#Determining the Colors of the Labels and Tickmarks
#If the user chose to save the 2D plot as a PNG, the labels on the axes, tickmarks, and title will be black so they can 
#be read in their computer's image viewer
if savemenu.value=='Yes':
    axiscolor='black'  #Color of the labels on the axes
    tickcolor='black'  #Color of the tickmarks on the axes
    titlecolor='black' #Color of the plot title
    
#If the user chose to not save the 2D plot as a PNG, the labels on the axes, tickmarks, and title will be the color 
#assigned to the type of terrain the golf shot is being launced from
else:
    axiscolor=labelcolor  #Color of the labels on the axes
    tickcolor=labelcolor  #Color of the tickmarks on the axes
    titlecolor=labelcolor #Color of the plot title

#Creating the 2D Plot Itself
fig=plt.figure(figsize=(7,5))                                                                                                         #Creates a figure and determines its size
plt.title("Golf Ball Trajectory using a " +str(clubname)+ " from the " +str(terrainname)+ " in " +str(weathername), color=titlecolor) #Plot Title
plt.grid(color='black', linestyle='--', linewidth=1)                                                                                  #Changes the plot to a 2D plot with black, dashed gridlines
fig.patch.set_facecolor(bordercolor)                                                                                                  #Creates a border around the 2D plot

#Plotting each Trajectory with Selected Type of Spin
if spinmenu.value=='Backspin/Topspin':
    plt.plot(traj1[:,1]*unitconversion, traj1[:,2]*unitconversion, 'blue', label='Backspin')                   #Plots the Trajectory with Backspin
    plt.plot(traj1[-1,1]*unitconversion,traj1[-1,2]*unitconversion,'o',color='blue')                           #Plots the Landing Point with Backspin
    farthestdistance1=traj1[-1,-1]                                                                             #Assigns the final point of the Backspin trajectory to the variable farthestdistance1
    spinname1="Backspin"                                                                                       #Assigns the name of the type of spin being modeled to the variable spinname1
    
    plt.plot(traj2[:,1]*unitconversion, traj2[:,2]*unitconversion, 'darkorange', label='Topsin')               #Plots the Trajectory with Topspin
    plt.plot(traj2[-1,1]*unitconversion,traj2[-1,2]*unitconversion,'o',color='darkorange')                     #Plots the Landing Point with Topspin
    farthestdistance2=traj2[-1,-1]                                                                             #Assigns the final point of the Topspin trajectory to the variable farthestdistance2
    spinname2="Topspin"                                                                                        #Assigns the name of the type of spin being modeled to the variable spinname2

if spinmenu.value=='Sidespin':
    plt.plot(traj3[:,1]*unitconversion, traj3[:,2]*unitconversion, 'red', label='Clockwise Sidespin')          #Plots the Trajectory with Clockwise Sidespin
    plt.plot(traj3[-1,1]*unitconversion,traj3[-1,2]*unitconversion,'o',color='red')                            #Plots the Landing Point with Clockwise Sidespin
    farthestdistance1=traj3[-1,-1]                                                                             #Assigns the final point of the Clockwise Sidespin trajectory to the variable farthestdistance1
    spinname1="Clockwise Sidespin"                                                                             #Assigns the name of the type of spin being modeled to the variable spinname1
    
    plt.plot(traj4[:,1]*unitconversion, traj4[:,2]*unitconversion, 'green', label='Counterclockwise Sidespin') #Plots the Trajectory with Counterclockwise Sidespin
    plt.plot(traj4[-1,1]*unitconversion,traj4[-1,2]*unitconversion,'o',color='green')                          #Plots the Landing Point with Counterclockwise Sidespin
    farthestdistance2=traj4[-1,-1]                                                                             #Assigns the final point of the Counterclockwise Sidespin trajectory to the variable farthestdistance2
    spinname2="Counterclockwise Sidespin"                                                                      #Assigns the name of the type of spin being modeled to the variable spinname2

if spinmenu.value=='Spiral':
    plt.plot(traj5[:,1]*unitconversion, traj5[:,2]*unitconversion, 'purple', label='Clockwise Spiral')         #Plots the Trajectory with Clockwise Spiral
    plt.plot(traj5[-1,1]*unitconversion,traj5[-1,2]*unitconversion,'o',color='purple')                         #Plots the Landing Point with Clockwise Spiral
    farthestdistance1=traj5[-1,1]                                                                              #Assigns the final point of the Clockwise Spiral trajectory to the variable farthestdistance1
    spinname1="Clockwise Spiral"                                                                               #Assigns the name of the type of spin being modeled to the variable spinname1
    
    plt.plot(traj6[:,1]*unitconversion, traj6[:,2]*unitconversion, 'yellow', label='Counterclockwise Spiral')  #Plots the Trajectory with Counterclockwise Spiral
    plt.plot(traj6[-1,1]*unitconversion,traj6[-1,2]*unitconversion,'o',color='yellow')                         #Plots the Landing Point with Counterclockwise Spiral
    farthestdistance2=traj6[-1,1]                                                                              #Assigns the final point of the Counterclockwise Spiral trajectory to the variable farthestdistance2
    spinname2="Counterclockwise Spiral"                                                                        #Assigns the name of the type of spin being modeled to the variable spinname2

#Determining if the User wanted to see the Hole on the Green and Plotting it if they Wanted to See it
if holemenu.value=='Yes':
        plt.plot(holeslider.value/holeconversion,0,'o',color='black',label='Hole')

#Labelling Axes
ax=plt.axes() #Converts plt commands to ax commands, which allows for the tickmarks of the 2D plot to be editted

plt.xlabel(unitlabelx, color=axiscolor)    #Labels the x-axis and assigns the color of the label
ax.tick_params(axis='x', colors=tickcolor) #Assigns tickmarks to the x-axis and colors the tickmarks

plt.ylabel(unitlabely, color=axiscolor)    #Labels the y-axis and assigns the color of the label
ax.tick_params(axis='y', colors=tickcolor) #Assigns tickmarks to the y-axis and colors the tickmarks

#Plotting the Legend
plt.legend(loc="best") #Plots the legend in the best location of the plot
plt.show()             #Displays the plot

#Saving the Image of the 2D Plot if the User wanted to Save a PNG of the 2D Plot
if savemenu.value=='Yes':                                                      
    plt.savefig(f'TITUS_Simulation_2D_Plot.png', dpi=500, bbox_inches='tight') 
    
#Creating the 3D Plot------------------------------------------------------------------------------------------------------
#Creating the 3D Plot Itself
fig=plt.figure(figsize=(7,5))                                                                                                      #Creates a figure and determines its size
ax=fig.add_subplot(111, projection='3d')                                                                                           #Changes the plot to be 3D
plt.title("Golf Ball Trajectory using a " +str(clubname)+ " from the " +str(terrainname)+ " in " +str(weathername), color='black') #Plot Title
fig.patch.set_facecolor(bordercolor)                                                                                               #Creates a border around the 3D plot

#Plotting each Trajectory with Selected Type of Spin
if spinmenu.value=='Backspin/Topspin':
    ax.plot(traj1[:,1]*unitconversion,traj1[:,2]*unitconversion,traj1[:,3]*unitconversion,zdir='y',color='blue', label='Backspin')
    ax.plot(traj2[:,1]*unitconversion,traj2[:,2]*unitconversion,traj2[:,3]*unitconversion,zdir='y',color='darkorange', label='Topspin')
if spinmenu.value=='Sidespin':
    ax.plot(traj3[:,1]*unitconversion,traj3[:,2]*unitconversion,traj3[:,3]*unitconversion,zdir='y',color='red', label='Clockwise Sidespin')
    ax.plot(traj4[:,1]*unitconversion,traj4[:,2]*unitconversion,traj4[:,3]*unitconversion,zdir='y',color='green', label='Counterclockwise Sidespin')
if spinmenu.value=='Spiral':
    ax.plot(traj5[:,1]*unitconversion,traj5[:,2]*unitconversion,traj5[:,3]*unitconversion,zdir='y',color='yellow', label='Clockwise Spiral')
    ax.plot(traj6[:,1]*unitconversion,traj6[:,2]*unitconversion,traj6[:,3]*unitconversion,zdir='y',color='purple', label='Counterclockwise Spiral')
    
#Labelling the Axes and Determining the Colors of the Labels and Tickmarks
ax.set_xlabel(unitlabelx, color='black') #Coloring the labels on the x-axis
ax.set_ylabel(unitlabelz, color='black') #Coloring the labels on the z-axis
ax.set_zlabel(unitlabely, color='black') #Coloring the labels on the y-axis
           
#Plotting the Legend
ax.legend(loc="lower right") #Plots the legend in the lower right corner of the 3D plot

#Coloring the Planes of the 3D Plot
plt.rcParams['grid.color']='black'    #Color of the gridlines
plt.rcParams['grid.linewidth']=1      #Thickness of the gridlines
ax.w_xaxis.pane.set_color(planecolor) #Colors the sky
ax.w_yaxis.pane.set_color(planecolor) #Colors the sky 
ax.w_zaxis.pane.set_color(mapcolor)   #Sets the ground to be the color of the terrain the user chose

#Printing Some Useful Data-------------------------------------------------------------------------------------------------
#Printing the Initially Input Conditions
print(u"\u001b[32;1m\u001b[4mINITIAL CONDITIONS\u001b[0m")
print("\033[4mSpinrate:\033[0m",spinrate,"rotations per minute")
print("\033[4mInitial Speed:\033[0m",format(vmag0mph/velocityconversion,'.2f'),unitvelocity)
print("\033[4mLaunch Angle in Y-Axis:\033[0m",format(UDangleslider.value,'.2f'),"\u00B0")
print("\033[4mLaunch Angle in Z-Axis:\033[0m",format(LRangleslider.value,'.2f'),"\u00B0")
print()

#Printing the Intitially Input Wind Conditions
print(u"\u001b[32;1m\u001b[4mWIND CONDITIONS\u001b[0m")

#Wind Conditions in X-Axis
if vwindx>0:
    print("\033[4mWind in X-Axis:\033[0m",format(vwindx/velocityconversion,'.2f'), unitvelocity, "backwards")
if vwindx<0:
    print("\033[4mWind in X-Axis:\033[0m",format(-vwindx/velocityconversion,'.2f'), unitvelocity, "forwards")
if vwindx==0:
    print("\033[4mWind in X-Axis:\033[0m NONE")

#Wind Conditions in Y-Axis
if vwindy>0:
    print("\033[4mWind in Y-Axis:\033[0m",format(vwindy/velocityconversion,'.2f'), unitvelocity, "downwards")
if vwindy<0:
    print("\033[4mWind in Y-Axis:\033[0m",format(-vwindy/velocityconversion,'.2f'), unitvelocity, "upwards")
if vwindy==0:
    print("\033[4mWind in Y-Axis:\033[0m NONE")
    
#Wind Conditions in Z-Axis
if vwindz>0:
    print("\033[4mWind in Z-Axis:\033[0m",format(vwindz/velocityconversion,'.2f'), unitvelocity, "left")
if vwindz<0:
    print("\033[4mWind in Z-Axis:\033[0m",format(-vwindz/velocityconversion,'.2f'), unitvelocity, "right")
if vwindz==0:
    print("\033[4mWind in Z-Axis:\033[0m NONE")
print()

#Printing the Maximum Distances based on Each Type of Spin
if spinmenu.value=='Backspin/Topspin':
    print(u"\u001b[32;1m\u001b[4mMAXIMUM DISTANCE\u001b[0m")
    print("\033[4mMaximum Distance Reached with Backspin:\033[0m",format(maxxback*unitconversion,'.2f'),unit)
    print("\033[4mMaximum Distance Reached with Topspin:\033[0m", format(maxxtop*unitconversion,'.2f'),unit)
    print()
    print(u"\u001b[32;1m\u001b[4mMAXIMUM HEIGHT\u001b[0m")
    print("\033[4mMaximum Height Reached with Backspin:\033[0m",format(maxyback*unitconversion,'.2f'),unit)
    print("\033[4mMaxiumum Height Reached with Topspin:\033[0m",format(maxytop*unitconversion,'.2f'),unit)
    print()
    print(u"\u001b[32;1m\u001b[4mMAXIMUM CURVE\u001b[0m")
    print("\033[4mMaximum Curve Reached with Backspin:\033[0m",format(maxzback*unitconversion,'.2f'),unit)
    print("\033[4mMaximum Curve Reached with Topspin:\033[0m",format(maxztop*unitconversion,'.2f'),unit)

if spinmenu.value=='Sidespin':
    print(u"\u001b[32;1m\u001b[4mMAXIMUM DISTANCE\u001b[0m")
    print("\033[4mMaximum Distance Reached with Clockwise Sidespin:\033[0m",format(maxxCLside*unitconversion,'.2f'),unit)
    print("\033[4mMaximum Distance Reached with Counterclockwise Sidespin:\033[0m",format(maxxCCside*unitconversion,'.2f'),unit)
    print()
    print(u"\u001b[32;1m\u001b[4mMAXIMUM HEIGHT\u001b[0m")
    print("\033[4mMaximum Height Reached with Clockwise Sidespin:\033[0m",format(maxyCLside*unitconversion,'.2f'),unit)
    print("\033[4mMaxiumum Height Reached with Counterclockwise Sidespin:\033[0m",format(maxyCCside*unitconversion,'.2f'),unit)
    print()
    print(u"\u001b[32;1m\u001b[4mMAXIMUM CURVE\u001b[0m")
    print("\033[4mMaximum Curve Reached with Clockwise Sidespin:\033[0m",format(maxzCLside*unitconversion,'.2f'),unit)
    print("\033[4mMaximum Curve Reached with Counterclockwise Sidespin:\033[0m",format(maxzCCside*unitconversion,'.2f'),unit)

if spinmenu.value=='Spiral':
    print(u"\u001b[32;1m\u001b[4mMAXIMUM DISTANCE\u001b[0m")
    print("\033[4mMaximum Distance Reached with Clockwise Spiral:\033[0m",format(maxxCLspiral*unitconversion,'.2f'),unit)
    print("\033[4mMaximum Distance Reached with Counterclockwise Spiral:\033[0m",format(maxxCCspiral*unitconversion,'.2f'),unit)
    print()
    print(u"\u001b[32;1m\u001b[4mMAXIMUM HEIGHT\u001b[0m")
    print("\033[4mMaximum Height Reached with Clockwise Spiral:\033[0m",format(maxyCLspiral*unitconversion,'.2f'),unit)
    print("\033[4mMaxiumum Height Reached with Counterclockwise Spiral:\033[0m",format(maxyCCspiral*unitconversion,'.2f'),unit)
    print()
    print(u"\u001b[32;1m\u001b[4mMAXIMUM CURVE\u001b[0m")
    print("\033[4mMaximum Curve Reached with Clockwise Spiral:\033[0m",format(maxzCLspiral*unitconversion,'.2f'),unit)
    print("\033[4mMaximum Curve Reached with Counterclockwise Spiral:\033[0m",format(maxzCCspiral*unitconversion,'.2f'),unit)
print()

#Printing both the Maximum Distance and Remaining Distance to the Hole after the Two Modeled Shots Land if the Hole was Plotted
if holemenu.value=='Yes':
    print(u"\u001b[32;1m\u001b[4mLANDING CONDITIONS\u001b[0m")
    print("\033[4mDistance to Hole from Starting Point:\033[0m",format(holeslider.value/holeconversion,'.0f'),unit)
    print("\033[4mDistance to Hole Remaining after Shot Landed with " +str(spinname1)+ ":\033[0m",format((holeslider.value/holeconversion)-(farthestdistance1*yardconversion),'.2f'),unit)
    print("\033[4mDistance to Hole Remaining after Shot Landed with " +str(spinname2)+ ":\033[0m",format((holeslider.value/holeconversion)-(farthestdistance2*yardconversion),'.2f'),unit)    

#Ending Message------------------------------------------------------------------------------------------------------------
display(Markdown('**<center><span style="color:green">Thank you for using the T.I.T.U.S., Mk. II! I hope this program was both fun and easy to use, and (more importantly) accurate to how an actual golf ball would move based on the given input parameters. Again, thank you for using this program and have a good day!</span></center>**'))
display(Markdown('**<center><span style="color:green">================================================= PROGRAM TERMINATED =================================================</span></center>**'))

## Validation
This program implements many factors in the math, so it may be difficult to keep track of what everything does. In order to see the validity of this program, one can consider the physics of what is actually happening. Balls with topspin will travel significantly shorter distances compared to balls with either no spin or backspin. Also, topspin results in a downward force on the ball, causing it to drop more quickly. Balls with backspin will travel further and expirience an upward force, giving it lift. Therefore, balls with backspin travel both further and higher than topspin. You can see this in both the 2D and 3D models, where the blue backspin lines are much larger in the x-axis than their topspin counterpart. Additionally, if the user implements wind, it can be seen in the 3D plot. Wind in specific directions will impact the trajectory of the ball in various ways. For example, if the user implements 15 mph winds the z-axis, they will see a significant bend in the trajectory, especially within the xz-plane.

## Results
The models generated by this code produce clean, easy to read, and informative plots. As you can see, the models are acurate to how actual golf balls fly through the air with the addition of wind and air resistance. I hope that these plots, and this programas a whole, is useful in your golfing endeavors.

## References
I would like to acknoweldge Dr. Aaron Titus, professor of physics at High Point University, for supplying many parts of this code (such as the various functions, setup of the code including formatting omega, initial parameters, and plotting). Additionally, I would like to thank Dr. Titus for his aid and skills in debugging and teaching me new code both during the Computational Physics course for which this project was made and his office hours.

I also have a handful of additional resources I used in order to create this project. Those sources are listed below:
* The textbook used for my Computational Physics course, "Computational Modeling and Visualization of Phyical Systems with Python" by Jay Wang. Specifically, chapter 3: Realistic Projectile Motion with Air Resistance, pages 77-78, in which modelling golf drives with drag and the Magnus Effect, the lift coefficiant and the drag coefficient of golf balls are described
* The offical mplot3d website: https://matplotlib.org/stable/tutorials/toolkits/mplot3d.html
* The official matplotlib color guide: https://matplotlib.org/3.1.0/gallery/color/named_colors.html
* Various golf websites and databases for their resources on golf clubs, swing data, ball speeds and dimensions, etc.
    * https://blog.trackmangolf.com/trackman-average-tour-stats/
    * https://meilyngiftshop.com/qa/what-is-the-average-swing-speed-for-a-7-iron.html#:~:text=The%20average%20male%20amateur's%20swing,average%20distance%20of%20214%20yards.
    * https://swingmangolf.com/average-golf-swing-speed-chart-2/
    * https://www.lihaoyi.com/post/BuildyourownCommandLinewithANSIescapecodes.html